# Extracción de contenido con Python a partir de XML

<html><p><span style="font-size:x-small">En GitHub, los enlaces de la tabla de contenidos de un notebook Jupyter no funcionan. Para una versión con enlaces, abrir en <a href="https://nbviewer.org/github/HD-aula-Literatura/II-2-scraping/blob/main/02-extraccion-con-python/02-extraccion-desde-xml.ipynb">NbViewer</a>. Para una versión interactiva del notebook, abrir en Binder:</span></p></html>

- [Introducción: XML](#s-intro)
  - [Sintaxis XML](#ss-sintaxis)
- [Manipular un árbol XML](#s-manipular-xml)
  - [Extracción del contenido](#ss-extraccion-xml)
    - [Metadatos](#sss-metadatos)
    - [Texto del poema](#sss-texto)
  - [Guardar la información extraída en una dataframe](#ss-guardar-dataframe)

<a name="s-intro"></a>

# Introducción: XML

Como se vio en el capítulo, XML (eXtensible Markup Language) es un lenguaje utilizado para representar
información de forma estructurada. Hablamos de él aquí ya que algunos documentos obtenidos por scraping pueden estar en XML, sin embargo las tecnologías XML se usan más típicamente para manipular corpus ya limpios y que han sido representados en XML. Como el HTML, se basa en una estructura arbórea, expresable mendiante una cadena de texto, en la que el contenido está incluido dentro de pares de etiquetas anidadas, y las etiquetas de abertura pueden tener atributos. Mientras que en HTML el conjunto de etiquetas posible está predefinido, y sus funciones se refieren a la estructuración y presentación de documentos web, en XML cada persona puede definir sus propias etiquetas (es extensible) y crear su propia especificación de marcado. Una especificación XML muy utilizada en humanidades es TEI (*Text Encoding Initiative*).

Hay tecnologías específicas para manipular documentos XML, como XPath, XSLT o XQuery y entornos para este fin, com Oxygen o BaseX. OpenRefine tiene algunas funciones para manipular XML, presentadas en el [presente repositorio](../01-extraccion-con-openrefine/02-extraccion-desde-xml.md). 


<a name="ss-sintaxis"></a>

## Sintaxis

Como se ha visto, para representar el árbol XML mediante una cadena de texto, se envuelve el contenido con pares de etiquetas anidables. Hay una etiqueta inicial de la que las demás descienden, representa el nodo raíz, en nuestro [ejemplo](#ejemplo-xml) es `<poema>`. Las etiquetas de abertura pueden llevar atributos; el valor del atributo va entre comillas o apóstrofes. Puede haber también etiquetas vacías o de autocierre, como `<fecha/>` en el [ejemplo](#ejemplo-xml).


Como se ve en el [ejemplo](#ejemplo-xml), un "prólogo" puede preceder al nodo raíz, con la forma `<?xml version="1.0" encoding="UTF-8"?>`, en él se especifica la versión y la codificación de caracteres.

Como en HTML, en XML hay también unas entidades predefinidas, cuyo uso es obligatorio en XML. Son las siguientes:

|Carácter|Entidad|
|:---:|:---:|
|<|&amp;lt;|
|>|&amp;gt;|
|"|&amp;quot;|
|'|&amp;apos;|
|&|&amp;amp;|

Es necesario usar entidades para representar estos caracteres, porque estos caracteres, de forma literal (no como entidades) se usan ya para definir la sintaxis XML cuando representamos el árbol XML como cadena de texto. Como se ve en el [ejemplo](#ejemplo-xml) abajo, `<` abre una etiqueta y `>` la cierra. Las comillas o apóstrofes se usan para los valores de los atributos. Y `&` se usa justamente para definir entidades. Así que para escribir estos signos como caracteres, no como integrantes de la sintaxis, hay que usar las entidades.

XML permite usar comentarios, que son ignorados por el parseador (el analizador que manipula el árbol del documento). Los documentos se introducen entre `<!-- -->` (misma sintaxis que los comentarios en HTML). Como se ve en el [ejemplo](#ejemplo-xml), dentro de los comentarios no hace falta usar entidades XML.


<a name="frag-ejemplo-xml"></a>

**Ejemplo de documento expresado en XML**
<html><span style="font-size:small">Elaboración propia</span></html>

```xml
<?xml version="1.0" encoding="UTF-8"?>
<poema>
  <metadatos>
    <autoria>
      <nombre>Guillermo Carlos</nombre>  
      <apellido>Guillermos</apellido>
    </autoria>
    <!-- el título es "fruta & frigo's" -->  
    <titulo>fruta &amp; frigo&apos;s</titulo>
    <fecha/>  
  </metadatos>
  <texto>
    <titulo>fruta &amp; frigo&apos;s</titulo>
    <estrofa tipo="haiku">
      <verso>come la fruta</verso>
      <verso>del refrigerador</verso>
      <verso>muy deliciosa</verso>
    </estrofa>
  </texto>
</poema>

```

<a name="s-manipular-xml"></a>

# Manipular un árbol XML

Como se dijo, hay lenguajes muy potentes para esto, como XPath, XSLT y XQuery. Aquí nos limitamos a la manipulación con bibliotecas de Python.

Por paralelismo a lo que se ha hecho para le HTML, utilizaremos la biblioteca [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#id17), que también permite manipular XML aparte de HTML. Una biblioteca Python más específica para XML es [lxml](https://lxml.de/).

Wikisource también puede ofrecer su contenido en XML, a través de comandos de su API (Application Programming Interface o interfaz de programación de aplicaciones, es decir, conjunto de instrucciones para interaccionar con un programa a partir de otro programa).

Bajaremos con la API la página del poema ya visto en el capítulo, "Una viñeta" de Delmira Agustini. El URL es https://es.wikisource.org/w/api.php?action=parse&page=Una_vi%C3%B1eta&format=xml.

No se dan detalles sobre el código ya que todo se ha visto en el [notebook](01-extraccion-desde-html.ipynb) sobre scraping desde HTML. La única diferencia es que, para crear un árbol XML con BeautifulSoup, como segundo argumento debemos dar la palabra `xml`: `arbol = BeautifulSoup(contenido, 'xml')`

In [2]:
import requests
from bs4 import BeautifulSoup

url_pagina = "https://es.wikisource.org/w/api.php?action=parse&page=Una_vi%C3%B1eta&format=xml"
respuesta = requests.get(url_pagina)
contenido = respuesta.text
arbol = BeautifulSoup(contenido, "xml")

In [3]:
arbol

<?xml version="1.0" encoding="utf-8"?>
<api><parse displaytitle="Una viñeta" pageid="27121" revid="1193682" title="Una viñeta"><text xml:space="preserve">&lt;div class="mw-parser-output"&gt;&lt;style data-mw-deduplicate="TemplateStyles:r1188054"&gt;.mw-parser-output #headertemplate{text-align:center;padding:1em;background:#F4F0E5;border-top:1px solid #A0A0A0;border-bottom:1px solid #A0A0A0;margin:10px auto;display:table;width:95%}.mw-parser-output #headerprevious,.mw-parser-output #footerprevious{padding:0;display:table-cell;vertical-align:middle;width:16%}.mw-parser-output #headernext,.mw-parser-output #footernext{padding:0;display:table-cell;vertical-align:middle;width:16%}.mw-parser-output .prev-span{float:left;display:block;font-size:smaller}.mw-parser-output .prev-span-arrow{margin-right:0.5em;position:relative;top:1.5px}.mw-parser-output .middle-div{width:68%;display:table-cell;vertical-align:middle}.mw-parser-output .next-span{float:right;display:block;font-size:smaller}.mw-pars

<a name="ss-extraccion-xml"></a>

## Extracción de contenido

Como se ve al mostrar el resultado, ahora tenemos un árbol XML, proveniente de la cadena de texto que expresa el contenido de la página con XML. Si miramos este XML, nos interesan los fragmentos siguientes:


**Para el título**


```xml
<parse displaytitle="Una viñeta" pageid="27121" revid="1193682" title="Una viñeta">
```

**Para la autora**

```xml
<links><pl exists="" ns="106" xml:space="preserve">Autor:Delmira Agustini</pl></links>
```

**Para el texto**
```xml
<text xml:space="preserve"> ... </text>
```

En los casos del título y de la autora se ve claramente la localización de la información. En el caso del texto del poema, la API de Wikisource no da un campo específico con su contenido. Lo que hace es dar, dentro del elemento `<text>`, una cadena de texto que expresa una parte del contenido principal del cuerpo de la página en HTML (la parte central con el poema y algunos de sus metadatos). Así que debemos extraer esta cadena, crear un árbol HTML y extraer el texto del poema a partir de este. No es el uso más típico de XML pero nos permite combinar los conocimientos vistos en el capítulo

<a name="sss-metadatos"></a>

### Metadatos

Vamos a extraer el título y autora primero, asignándolos a una variable:

In [4]:
titulo = arbol.find("parse").attrs["title"]

In [5]:
print(titulo)

Una viñeta


In [6]:
autora = arbol.find_all("links")[0].find("pl").get_text().replace("Autor:", "")

In [7]:
print(autora)

Delmira Agustini


<a name="sss-texto"></a>

### Texto del poema

Nos ocupamos ahora del texto. Primero extraemos la parte central del cuerpo de la página Wikisource que contiene el poema con alguna información más (como se ha dicho, la API no divide el contenido de forma más detallada).

In [8]:
contenido_cuerpo_pagina = arbol.find("text").get_text()

In [9]:
print(contenido_cuerpo_pagina)

<div class="mw-parser-output"><style data-mw-deduplicate="TemplateStyles:r1188054">.mw-parser-output #headertemplate{text-align:center;padding:1em;background:#F4F0E5;border-top:1px solid #A0A0A0;border-bottom:1px solid #A0A0A0;margin:10px auto;display:table;width:95%}.mw-parser-output #headerprevious,.mw-parser-output #footerprevious{padding:0;display:table-cell;vertical-align:middle;width:16%}.mw-parser-output #headernext,.mw-parser-output #footernext{padding:0;display:table-cell;vertical-align:middle;width:16%}.mw-parser-output .prev-span{float:left;display:block;font-size:smaller}.mw-parser-output .prev-span-arrow{margin-right:0.5em;position:relative;top:1.5px}.mw-parser-output .middle-div{width:68%;display:table-cell;vertical-align:middle}.mw-parser-output .next-span{float:right;display:block;font-size:smaller}.mw-parser-output .next-span-arrow{margin-left:0.5em;position:relative;top:1.5px}.mw-parser-output .lower-div{clear:both;padding:0.5em;text-align:center;margin:0 auto;font-si

Ya que esta cadena expresa el HTML de la página, la analizaremos con el parseador de HTML y extraeremos el texto del poema, usando las mismas operaciones que ya se vieron en el [notebook](01-extraccion-desde-html.ipynb) sobre scraping desde HTML.

In [12]:
arbol_html = BeautifulSoup(contenido_cuerpo_pagina)

In [13]:
texto_bruto_poema = arbol_html.find_all("div", attrs={"class": "poem"})[0].find("p").get_text()

In [14]:
texto_bruto_poema

'\xa0Tarde sucia de invierno. El caserío,   \n\xa0como si fuera un croquis al creyón,   \n\xa0se hunde en la noche. El humo de un bohío,   \n\xa0que sube en forma de tirabuzón;   \n\xa0\n\n\xa0mancha el paisaje que produce frío,  \n\xa0y debajo de la genuflexión   \n\xa0de la arboleda, somormuja el río   \n\xa0su canción, su somnífera canción.   \n\xa0\n\n\xa0Los labradores, camellón abajo,   \n\xa0retornan fatigosos del trabajo,  \n\xa0como un problema sin definición.   \n\xa0\n\n\xa0Y el dueño del terruño, indiferente,   \n\xa0rápidamente, muy rápidamente,   \n\xa0baja en su coche por el camellón.\n'

In [15]:
versos_brutos = texto_bruto_poema.split("\n")

In [16]:
versos_brutos

['\xa0Tarde sucia de invierno. El caserío,   ',
 '\xa0como si fuera un croquis al creyón,   ',
 '\xa0se hunde en la noche. El humo de un bohío,   ',
 '\xa0que sube en forma de tirabuzón;   ',
 '\xa0',
 '',
 '\xa0mancha el paisaje que produce frío,  ',
 '\xa0y debajo de la genuflexión   ',
 '\xa0de la arboleda, somormuja el río   ',
 '\xa0su canción, su somnífera canción.   ',
 '\xa0',
 '',
 '\xa0Los labradores, camellón abajo,   ',
 '\xa0retornan fatigosos del trabajo,  ',
 '\xa0como un problema sin definición.   ',
 '\xa0',
 '',
 '\xa0Y el dueño del terruño, indiferente,   ',
 '\xa0rápidamente, muy rápidamente,   ',
 '\xa0baja en su coche por el camellón.',
 '']

In [17]:
versos_limpios = [verso.strip() for verso in versos_brutos]

In [18]:
versos_limpios

['Tarde sucia de invierno. El caserío,',
 'como si fuera un croquis al creyón,',
 'se hunde en la noche. El humo de un bohío,',
 'que sube en forma de tirabuzón;',
 '',
 '',
 'mancha el paisaje que produce frío,',
 'y debajo de la genuflexión',
 'de la arboleda, somormuja el río',
 'su canción, su somnífera canción.',
 '',
 '',
 'Los labradores, camellón abajo,',
 'retornan fatigosos del trabajo,',
 'como un problema sin definición.',
 '',
 '',
 'Y el dueño del terruño, indiferente,',
 'rápidamente, muy rápidamente,',
 'baja en su coche por el camellón.',
 '']

In [19]:
texto_poema = "\n".join(versos_limpios)

In [20]:
texto_poema

'Tarde sucia de invierno. El caserío,\ncomo si fuera un croquis al creyón,\nse hunde en la noche. El humo de un bohío,\nque sube en forma de tirabuzón;\n\n\nmancha el paisaje que produce frío,\ny debajo de la genuflexión\nde la arboleda, somormuja el río\nsu canción, su somnífera canción.\n\n\nLos labradores, camellón abajo,\nretornan fatigosos del trabajo,\ncomo un problema sin definición.\n\n\nY el dueño del terruño, indiferente,\nrápidamente, muy rápidamente,\nbaja en su coche por el camellón.\n'

In [21]:
print(texto_poema)

Tarde sucia de invierno. El caserío,
como si fuera un croquis al creyón,
se hunde en la noche. El humo de un bohío,
que sube en forma de tirabuzón;


mancha el paisaje que produce frío,
y debajo de la genuflexión
de la arboleda, somormuja el río
su canción, su somnífera canción.


Los labradores, camellón abajo,
retornan fatigosos del trabajo,
como un problema sin definición.


Y el dueño del terruño, indiferente,
rápidamente, muy rápidamente,
baja en su coche por el camellón.



<a name="ss-guardar-dataframe"></a>

## Guardar la información extraída en una dataframe

Gracias a la extracción, tenemos la información que nos interesa en las variables `titulo`, `autora` y `texto_poema`.

Podemos guardar esto en una dataframe por ejemplo, como se vio en el [notebook](01-extraccion-desde-html.ipynb) sobre scraping desde HTML.

In [22]:
import pandas as pd

datos = {"autora": [autora],
         "titulo": [titulo],
         "textoPoema": [texto_poema]} 

df = pd.DataFrame(datos)

In [23]:
df

,autora,titulo,textoPoema
0,Delmira Agustini,Una viñeta,"Tarde sucia de invierno. El caserío,\ncomo si ..."


Podemos ahora guardar este archivo en el disco.

In [24]:
import pathlib

# definir directorio para el archivo de salida
directorio_actual = pathlib.Path.cwd()
dir_resultados = directorio_actual.joinpath("resultados", "desde-xml")

# crear el directorio (si ya existe no será creado)
if not dir_resultados.is_dir():
    dir_resultados.mkdir(parents=True)

# preparar los nombres de los archivos de salida para cada formato
nombre_fichero = "agustini_poema"
salida_csv = dir_resultados.joinpath(nombre_fichero + ".csv")
salida_tsv = dir_resultados.joinpath(nombre_fichero + ".tsv")
salida_excel = dir_resultados.joinpath(nombre_fichero + ".xlsx")

In [25]:
df.to_csv(salida_csv, index=False)
df.to_csv(salida_tsv, sep="\t", index=False)
df.to_excel(salida_excel, index=False)

Abriendo el archivo .xlsx con LibreOffice Calc, el resultado se muestra como sigue:


![img](./img/02_xml_01_excel.png)